In [1]:
import numpy as np
import pandas as pd
import sys


sys.path.append("../dora/models")
from utils import read_data, process_time, merge_data, dataset_builder, cumulative_sale_by_category

NUMBER_OF_LAGS = 4

from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm

In [2]:
infos, items, orders = read_data("../main/datasets/")
process_time(orders)
df = dataset_builder(orders, items)

### put here the parameters

In [3]:
path_to_pred = "pred/lgbm_kaggle_df.csv"
path_to_arrima_cat3 = "pred/arrima_category2.csv"
CATEGORY = "category2"

In [4]:
orderSum_cat_arrima = pd.read_csv(path_to_arrima_cat3,sep="|")
pred = pd.read_csv(path_to_pred,sep="|")

In [5]:
#adding CATEGORY back
pred2 = pd.merge(pred,items[['itemID',CATEGORY]],how='left',on='itemID',validate="m:1")

#getting current total sales of CATEGORY
orderSum_cat_cur = pred2.groupby(CATEGORY).demandPrediction.sum()
orderSum_cat_cur.name= 'orderSum_cat_cur'
pred2 = pd.merge(pred2, orderSum_cat_cur, how='left',on=CATEGORY,validate="m:1")

#getting arrima prediction of total sales of CATEGORY
pred2 = pd.merge(pred2, orderSum_cat_arrima, how='left',on=CATEGORY,validate="m:1")

#percent of sales of the itemID in the CATEGORY
pred2['percent'] = (pred2.demandPrediction / pred2.orderSum_cat_cur)


#scaling to the arrima predction
pred2['demandPrediction'] = pred2.percent * pred2.orderSum_cat_arrima
pred2['demandPrediction'].fillna(0,inplace=True)
pred2 = pred2[['itemID','demandPrediction']]

In [6]:
def baseline_score(prediction, target, simulatedPrice):
    prediction = prediction.astype(int)

    return np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice)

In [7]:
result = df.query('group_backwards == 1')
result.sort_values('itemID', inplace=True)

print(baseline_score(pred['demandPrediction'], result['orderSum'].values, infos['simulationPrice']))
print(baseline_score(pred2['demandPrediction'], result['orderSum'].values, infos['simulationPrice']))

83109.12599999999
-2609856.2140000006


/home/g/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# tentei para category2 e brand, todos deram resultados muito piores que antes